In [26]:
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import KFold

In [27]:
# Verisetinden verilerin okunması
covid = pd.read_csv("covid_dataset.csv")
covid.head()

,Breathing Problem,Fever,Dry Cough,Sore throat,Running Nose,Asthma,Chronic Lung Disease,Headache,Heart Disease,Diabetes,...,Fatigue,Gastrointestinal,Abroad travel,Contact with COVID Patient,Attended Large Gathering,Visited Public Exposed Places,Family working in Public Exposed Places,Wearing Masks,Sanitization from Market,COVID-19
0,Yes,Yes,Yes,Yes,Yes,No,No,No,No,Yes,...,Yes,Yes,No,Yes,No,Yes,Yes,No,No,Yes
1,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,No,No,...,Yes,No,No,No,Yes,Yes,No,No,No,Yes
2,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,...,Yes,Yes,Yes,No,No,No,No,No,No,Yes
3,Yes,Yes,Yes,No,No,Yes,No,No,Yes,Yes,...,No,No,Yes,No,Yes,Yes,No,No,No,Yes
4,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,...,No,Yes,No,Yes,No,Yes,No,No,No,Yes


In [28]:
# Verisetindeki tüm örneklerde 0 değerine sahip olan gereksiz özniteliklerin çıkarılması.
covid=covid.drop('Wearing Masks',axis=1)
covid=covid.drop('Sanitization from Market',axis=1)

In [29]:
# Verilerin 1 ve 0'lara dönüştürülmesi.
from sklearn.preprocessing import LabelEncoder
covid = covid.apply(LabelEncoder().fit_transform)

In [30]:
covid.head()

,Breathing Problem,Fever,Dry Cough,Sore throat,Running Nose,Asthma,Chronic Lung Disease,Headache,Heart Disease,Diabetes,Hyper Tension,Fatigue,Gastrointestinal,Abroad travel,Contact with COVID Patient,Attended Large Gathering,Visited Public Exposed Places,Family working in Public Exposed Places,COVID-19
0,1,1,1,1,1,0,0,0,0,1,1,1,1,0,1,0,1,1,1
1,1,1,1,1,0,1,1,1,0,0,0,1,0,0,0,1,1,0,1
2,1,1,1,1,1,1,1,1,0,1,0,1,1,1,0,0,0,0,1
3,1,1,1,0,0,1,0,0,1,1,0,0,0,1,0,1,1,0,1
4,1,1,1,1,1,0,1,1,1,1,1,0,1,0,1,0,1,0,1


In [31]:
# Üzerinde sınıflandırma yaptığımız COVID-19 özniteliği verilerinin y_total değişkenine, diğer öznitelik verilerinin ise X_total değişkenine atanması.
X_total = covid.iloc[:, 0:18].values
y_total = covid.iloc[:,18].values

In [34]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
print("\nNaive Bayes")

kf = KFold(n_splits=10, random_state=20, shuffle=True)
avg_acc, avg_rec, avg_f1, avg_prec, avg_auc, avg_spec, avg_mcc, iterasyon = 0, 0, 0, 0, 0, 0, 0, 0
print('\n===========================================')

for train_index, test_index in kf.split(X_total):
    X_train, X_test = X_total[train_index], X_total[test_index] 
    y_train, y_test = y_total[train_index], y_total[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    cm=confusion_matrix(y_pred,y_test)
    cm = np.fliplr(cm)
    cm = cm[::-1]
    iterasyon = iterasyon +1
    print(f"\nİterasyon: {iterasyon}")
    print('\nconfusion_matrix:\n',cm)
    print(f"\nAccuracy Score: {accuracy_score(y_test, y_pred):.3f}")
    print(f"Precision Score: {precision_score(y_test, y_pred):.3f}")
    print(f"Recall Score: {recall_score(y_test, y_pred):.3f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.3f}")
    print(f"Auc Score: {roc_auc_score(y_test, y_pred):.3f}")
    print(f"Specificity Score: {cm[1][1]/(cm[1][1]+cm[0][1]):.3f}")
    print(f"Mcc: {matthews_corrcoef(y_test, y_pred):.3f}")
    print('\n===========================================')
    avg_acc += accuracy_score(y_test,y_pred)
    avg_rec += recall_score(y_test,y_pred)
    avg_prec += precision_score(y_test,y_pred)
    avg_f1 += f1_score(y_test,y_pred)
    avg_auc += roc_auc_score(y_test,y_pred)
    avg_spec += cm[1][1]/(cm[1][1]+cm[0][1])
    avg_mcc += matthews_corrcoef(y_test, y_pred)
    
print(f"\nAvg. Accuracy: {avg_acc/10:.3f}")
print(f"Avg. Precision: {avg_prec/10:.3f}")
print(f"Avg. Recall: {avg_rec/10:.3f}")
print(f"Avg. F1Score: {avg_f1/10:.3f}")
print(f"Avg. Auc: {avg_auc/10:.3f}")
print(f"Avg. Specificity: {avg_spec/10:.3f}")
print(f"Avg. MCC: {avg_mcc/10:.3f}")
print('\n===========================================')



Naive Bayes


İterasyon: 1

confusion_matrix:
 [[290   0]
 [145 109]]

Accuracy Score: 0.733
Precision Score: 1.000
Recall Score: 0.667
F1 Score: 0.800
Auc Score: 0.833
Specificity Score: 1.000
Mcc: 0.535


İterasyon: 2

confusion_matrix:
 [[314   0]
 [123 107]]

Accuracy Score: 0.774
Precision Score: 1.000
Recall Score: 0.719
F1 Score: 0.836
Auc Score: 0.859
Specificity Score: 1.000
Mcc: 0.578


İterasyon: 3

confusion_matrix:
 [[308   0]
 [133 103]]

Accuracy Score: 0.756
Precision Score: 1.000
Recall Score: 0.698
F1 Score: 0.822
Auc Score: 0.849
Specificity Score: 1.000
Mcc: 0.552


İterasyon: 4

confusion_matrix:
 [[312   0]
 [133  99]]

Accuracy Score: 0.756
Precision Score: 1.000
Recall Score: 0.701
F1 Score: 0.824
Auc Score: 0.851
Specificity Score: 1.000
Mcc: 0.547


İterasyon: 5

confusion_matrix:
 [[313   0]
 [134  96]]

Accuracy Score: 0.753
Precision Score: 1.000
Recall Score: 0.700
F1 Score: 0.824
Auc Score: 0.850
Specificity Score: 1.000
Mcc: 0.541


İterasyon: 6

confus

In [35]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression() 
print("\nLogistic Regression")  

kf = KFold(n_splits=10, random_state=20, shuffle=True)
avg_acc, avg_rec, avg_f1, avg_prec, avg_auc, avg_spec, avg_mcc, iterasyon = 0, 0, 0, 0, 0, 0, 0, 0
print('\n===========================================')
for train_index, test_index in kf.split(X_total):
    X_train, X_test = X_total[train_index], X_total[test_index] 
    y_train, y_test = y_total[train_index], y_total[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    cm=confusion_matrix(y_pred,y_test)
    cm = np.fliplr(cm)
    cm = cm[::-1]
    iterasyon = iterasyon +1
    print(f"\nİterasyon: {iterasyon}")
    print('\nconfusion_matrix:\n',cm)
    print(f"\nAccuracy Score: {accuracy_score(y_test, y_pred):.3f}")
    print(f"Precision Score: {precision_score(y_test, y_pred):.3f}")
    print(f"Recall Score: {recall_score(y_test, y_pred):.3f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.3f}")
    print(f"Auc Score: {roc_auc_score(y_test, y_pred):.3f}")
    print(f"Specificity Score: {cm[1][1]/(cm[1][1]+cm[0][1]):.3f}")
    print(f"Mcc: {matthews_corrcoef(y_test, y_pred):.3f}")
    print('\n===========================================')
    avg_acc += accuracy_score(y_test,y_pred)
    avg_rec += recall_score(y_test,y_pred)
    avg_prec += precision_score(y_test,y_pred)
    avg_f1 += f1_score(y_test,y_pred)
    avg_auc += roc_auc_score(y_test,y_pred)
    avg_spec += cm[1][1]/(cm[1][1]+cm[0][1])
    avg_mcc += matthews_corrcoef(y_test, y_pred)
    
print(f"\nAvg. Accuracy: {avg_acc/10:.3f}")
print(f"Avg. Precision: {avg_prec/10:.3f}")
print(f"Avg. Recall: {avg_rec/10:.3f}")
print(f"Avg. F1 Score: {avg_f1/10:.3f}")
print(f"Avg. Auc: {avg_auc/10:.3f}")
print(f"Avg. Specificity: {avg_spec/10:.3f}")
print(f"Avg. MCC: {avg_mcc/10:.3f}")
print('\n===========================================')



Logistic Regression


İterasyon: 1

confusion_matrix:
 [[429  14]
 [  6  95]]

Accuracy Score: 0.963
Precision Score: 0.968
Recall Score: 0.986
F1 Score: 0.977
Auc Score: 0.929
Specificity Score: 0.872
Mcc: 0.883


İterasyon: 2

confusion_matrix:
 [[433  12]
 [  4  95]]

Accuracy Score: 0.971
Precision Score: 0.973
Recall Score: 0.991
F1 Score: 0.982
Auc Score: 0.939
Specificity Score: 0.888
Mcc: 0.905


İterasyon: 3

confusion_matrix:
 [[437  17]
 [  4  86]]

Accuracy Score: 0.961
Precision Score: 0.963
Recall Score: 0.991
F1 Score: 0.977
Auc Score: 0.913
Specificity Score: 0.835
Mcc: 0.871


İterasyon: 4

confusion_matrix:
 [[434   7]
 [ 11  92]]

Accuracy Score: 0.967
Precision Score: 0.984
Recall Score: 0.975
F1 Score: 0.980
Auc Score: 0.952
Specificity Score: 0.929
Mcc: 0.891


İterasyon: 5

confusion_matrix:
 [[442  15]
 [  5  81]]

Accuracy Score: 0.963
Precision Score: 0.967
Recall Score: 0.989
F1 Score: 0.978
Auc Score: 0.916
Specificity Score: 0.844
Mcc: 0.870


İterasyon: 6

In [46]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=5)
print("\nkNN")

kf = KFold(n_splits=10, random_state=20, shuffle=True)
avg_acc, avg_rec, avg_f1, avg_prec, avg_auc, avg_spec, avg_mcc, iterasyon = 0, 0, 0, 0, 0, 0, 0, 0
print('\n===========================================')
for train_index, test_index in kf.split(X_total):
    X_train, X_test = X_total[train_index], X_total[test_index] 
    y_train, y_test = y_total[train_index], y_total[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    cm=confusion_matrix(y_pred,y_test)
    cm = np.fliplr(cm)
    cm = cm[::-1]
    iterasyon = iterasyon +1
    print(f"\nİterasyon: {iterasyon}")
    print('\nconfusion_matrix:\n',cm)
    print(f"\nAccuracy Score: {accuracy_score(y_test, y_pred):.3f}")
    print(f"Precision Score: {precision_score(y_test, y_pred):.3f}")
    print(f"Recall Score: {recall_score(y_test, y_pred):.3f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.3f}")
    print(f"Auc Score: {roc_auc_score(y_test, y_pred):.3f}")
    print(f"Specificity Score: {cm[1][1]/(cm[1][1]+cm[0][1]):.3f}")
    print(f"Mcc: {matthews_corrcoef(y_test, y_pred):.3f}")
    print('\n===========================================')
    avg_acc += accuracy_score(y_test,y_pred)
    avg_rec += recall_score(y_test,y_pred)
    avg_prec += precision_score(y_test,y_pred)
    avg_f1 += f1_score(y_test,y_pred)
    avg_auc += roc_auc_score(y_test,y_pred)
    avg_spec += cm[1][1]/(cm[1][1]+cm[0][1])
    avg_mcc += matthews_corrcoef(y_test, y_pred)
    
print(f"\nAvg. Accuracy: {avg_acc/10:.3f}")
print(f"Avg. Precision: {avg_prec/10:.3f}")
print(f"Avg. Recall: {avg_rec/10:.3f}")
print(f"Avg. F1Score: {avg_f1/10:.3f}")
print(f"Avg. Auc: {avg_auc/10:.3f}")
print(f"Avg. Specificity: {avg_spec/10:.3f}")
print(f"Avg. MCC: {avg_mcc/10:.3f}")
print('\n===========================================')



kNN


İterasyon: 1

confusion_matrix:
 [[428   5]
 [  7 104]]

Accuracy Score: 0.978
Precision Score: 0.988
Recall Score: 0.984
F1 Score: 0.986
Auc Score: 0.969
Specificity Score: 0.954
Mcc: 0.932


İterasyon: 2

confusion_matrix:
 [[428   1]
 [  9 106]]

Accuracy Score: 0.982
Precision Score: 0.998
Recall Score: 0.979
F1 Score: 0.988
Auc Score: 0.985
Specificity Score: 0.991
Mcc: 0.944


İterasyon: 3

confusion_matrix:
 [[433   9]
 [  8  94]]

Accuracy Score: 0.969
Precision Score: 0.980
Recall Score: 0.982
F1 Score: 0.981
Auc Score: 0.947
Specificity Score: 0.913
Mcc: 0.898


İterasyon: 4

confusion_matrix:
 [[438   2]
 [  7  97]]

Accuracy Score: 0.983
Precision Score: 0.995
Recall Score: 0.984
F1 Score: 0.990
Auc Score: 0.982
Specificity Score: 0.980
Mcc: 0.946


İterasyon: 5

confusion_matrix:
 [[441   5]
 [  6  91]]

Accuracy Score: 0.980
Precision Score: 0.989
Recall Score: 0.987
F1 Score: 0.988
Auc Score: 0.967
Specificity Score: 0.948
Mcc: 0.931


İterasyon: 6

confusion_matr

In [47]:
from sklearn import tree
model = tree.DecisionTreeClassifier()
print("\nDecision Tree")

kf = KFold(n_splits=10, random_state=20, shuffle=True)
avg_acc, avg_rec, avg_f1, avg_prec, avg_auc, avg_spec, avg_mcc, iterasyon = 0, 0, 0, 0, 0, 0, 0, 0
print('\n===========================================')
for train_index, test_index in kf.split(X_total):
    X_train, X_test = X_total[train_index], X_total[test_index] 
    y_train, y_test = y_total[train_index], y_total[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    cm=confusion_matrix(y_pred,y_test)
    cm = np.fliplr(cm)
    cm = cm[::-1]
    iterasyon = iterasyon +1
    print(f"\nİterasyon: {iterasyon}")
    print('\nconfusion_matrix:\n',cm)
    print(f"\nAccuracy Score: {accuracy_score(y_test, y_pred):.3f}")
    print(f"Precision Score: {precision_score(y_test, y_pred):.3f}")
    print(f"Recall Score: {recall_score(y_test, y_pred):.3f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.3f}")
    print(f"Auc Score: {roc_auc_score(y_test, y_pred):.3f}")
    print(f"Specificity Score: {cm[1][1]/(cm[1][1]+cm[0][1]):.3f}")
    print(f"Mcc: {matthews_corrcoef(y_test, y_pred):.3f}")
    print('\n===========================================')
    avg_acc += accuracy_score(y_test,y_pred)
    avg_rec += recall_score(y_test,y_pred)
    avg_prec += precision_score(y_test,y_pred)
    avg_f1 += f1_score(y_test,y_pred)
    avg_auc += roc_auc_score(y_test,y_pred)
    avg_spec += cm[1][1]/(cm[1][1]+cm[0][1])
    avg_mcc += matthews_corrcoef(y_test, y_pred)
    
print(f"\nAvg. Accuracy: {avg_acc/10:.3f}")
print(f"Avg. Precision: {avg_prec/10:.3f}")
print(f"Avg. Recall: {avg_rec/10:.3f}")
print(f"Avg. F1Score: {avg_f1/10:.3f}")
print(f"Avg. Auc: {avg_auc/10:.3f}")
print(f"Avg. Specificity: {avg_spec/10:.3f}")
print(f"Avg. MCC: {avg_mcc/10:.3f}")
print('\n===========================================')




Decision Tree


İterasyon: 1

confusion_matrix:
 [[429   8]
 [  6 101]]

Accuracy Score: 0.974
Precision Score: 0.982
Recall Score: 0.986
F1 Score: 0.984
Auc Score: 0.956
Specificity Score: 0.927
Mcc: 0.919


İterasyon: 2

confusion_matrix:
 [[428   1]
 [  9 106]]

Accuracy Score: 0.982
Precision Score: 0.998
Recall Score: 0.979
F1 Score: 0.988
Auc Score: 0.985
Specificity Score: 0.991
Mcc: 0.944


İterasyon: 3

confusion_matrix:
 [[429   2]
 [ 12 101]]

Accuracy Score: 0.974
Precision Score: 0.995
Recall Score: 0.973
F1 Score: 0.984
Auc Score: 0.977
Specificity Score: 0.981
Mcc: 0.921


İterasyon: 4

confusion_matrix:
 [[439   4]
 [  6  95]]

Accuracy Score: 0.982
Precision Score: 0.991
Recall Score: 0.987
F1 Score: 0.989
Auc Score: 0.973
Specificity Score: 0.960
Mcc: 0.939


İterasyon: 5

confusion_matrix:
 [[441   4]
 [  6  92]]

Accuracy Score: 0.982
Precision Score: 0.991
Recall Score: 0.987
F1 Score: 0.989
Auc Score: 0.972
Specificity Score: 0.958
Mcc: 0.937


İterasyon: 6

conf

In [69]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=5)
print("\nRandom Forest")

kf = KFold(n_splits=10, random_state=20, shuffle=True)
avg_acc, avg_rec, avg_f1, avg_prec, avg_auc, avg_spec, avg_mcc, iterasyon = 0, 0, 0, 0, 0, 0, 0, 0
print('\n===========================================')
for train_index, test_index in kf.split(X_total):
    X_train, X_test = X_total[train_index], X_total[test_index] 
    y_train, y_test = y_total[train_index], y_total[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    cm=confusion_matrix(y_pred,y_test)
    cm = np.fliplr(cm)
    cm = cm[::-1]
    iterasyon = iterasyon +1
    print(f"\nİterasyon: {iterasyon}")
    print('\nconfusion_matrix:\n',cm)
    print(f"\nAccuracy Score: {accuracy_score(y_test, y_pred):.3f}")
    print(f"Precision Score: {precision_score(y_test, y_pred):.3f}")
    print(f"Recall Score: {recall_score(y_test, y_pred):.3f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.3f}")
    print(f"Auc Score: {roc_auc_score(y_test, y_pred):.3f}")
    print(f"Specificity Score: {cm[1][1]/(cm[1][1]+cm[0][1]):.3f}")
    print(f"Mcc: {matthews_corrcoef(y_test, y_pred):.3f}")
    print('\n===========================================')
    avg_acc += accuracy_score(y_test,y_pred)
    avg_rec += recall_score(y_test,y_pred)
    avg_prec += precision_score(y_test,y_pred)
    avg_f1 += f1_score(y_test,y_pred)
    avg_auc += roc_auc_score(y_test,y_pred)
    avg_spec += cm[1][1]/(cm[1][1]+cm[0][1])
    avg_mcc += matthews_corrcoef(y_test, y_pred)
    
print(f"\nAvg. Accuracy: {avg_acc/10:.3f}")
print(f"Avg. Precision: {avg_prec/10:.3f}")
print(f"Avg. Recall: {avg_rec/10:.3f}")
print(f"Avg. F1Score: {avg_f1/10:.3f}")
print(f"Avg. Auc: {avg_auc/10:.3f}")
print(f"Avg. Specificity: {avg_spec/10:.3f}")
print(f"Avg. MCC: {avg_mcc/10:.3f}")
print('\n===========================================')




Random Forest


İterasyon: 1

confusion_matrix:
 [[435  23]
 [  0  86]]

Accuracy Score: 0.958
Precision Score: 0.950
Recall Score: 1.000
F1 Score: 0.974
Auc Score: 0.894
Specificity Score: 0.789
Mcc: 0.866


İterasyon: 2

confusion_matrix:
 [[437  16]
 [  0  91]]

Accuracy Score: 0.971
Precision Score: 0.965
Recall Score: 1.000
F1 Score: 0.982
Auc Score: 0.925
Specificity Score: 0.850
Mcc: 0.906


İterasyon: 3

confusion_matrix:
 [[440  24]
 [  1  79]]

Accuracy Score: 0.954
Precision Score: 0.948
Recall Score: 0.998
F1 Score: 0.972
Auc Score: 0.882
Specificity Score: 0.767
Mcc: 0.846


İterasyon: 4

confusion_matrix:
 [[445  15]
 [  0  84]]

Accuracy Score: 0.972
Precision Score: 0.967
Recall Score: 1.000
F1 Score: 0.983
Auc Score: 0.924
Specificity Score: 0.848
Mcc: 0.906


İterasyon: 5

confusion_matrix:
 [[447  18]
 [  0  78]]

Accuracy Score: 0.967
Precision Score: 0.961
Recall Score: 1.000
F1 Score: 0.980
Auc Score: 0.906
Specificity Score: 0.812
Mcc: 0.884


İterasyon: 6

conf

In [70]:
from sklearn import svm
model = svm.SVC(kernel='linear')
print("\nLinear SVM")

kf = KFold(n_splits=10, random_state=20, shuffle=True)
avg_acc, avg_rec, avg_f1, avg_prec, avg_auc, avg_spec, avg_mcc, iterasyon = 0, 0, 0, 0, 0, 0, 0, 0
print('\n===========================================')
for train_index, test_index in kf.split(X_total):
    X_train, X_test = X_total[train_index], X_total[test_index] 
    y_train, y_test = y_total[train_index], y_total[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    cm=confusion_matrix(y_pred,y_test)
    cm = np.fliplr(cm)
    cm = cm[::-1]
    iterasyon = iterasyon +1
    print(f"\nİterasyon: {iterasyon}")
    print('\nconfusion_matrix:\n',cm)
    print(f"\nAccuracy Score: {accuracy_score(y_test, y_pred):.3f}")
    print(f"Precision Score: {precision_score(y_test, y_pred):.3f}")
    print(f"Recall Score: {recall_score(y_test, y_pred):.3f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.3f}")
    print(f"Auc Score: {roc_auc_score(y_test, y_pred):.3f}")
    print(f"Specificity Score: {cm[1][1]/(cm[1][1]+cm[0][1]):.3f}")
    print(f"Mcc: {matthews_corrcoef(y_test, y_pred):.3f}")
    print('\n===========================================')
    avg_acc += accuracy_score(y_test,y_pred)
    avg_rec += recall_score(y_test,y_pred)
    avg_prec += precision_score(y_test,y_pred)
    avg_f1 += f1_score(y_test,y_pred)
    avg_auc += roc_auc_score(y_test,y_pred)
    avg_spec += cm[1][1]/(cm[1][1]+cm[0][1])
    avg_mcc += matthews_corrcoef(y_test, y_pred)
    
print(f"\nAvg. Accuracy: {avg_acc/10:.3f}")
print(f"Avg. Precision: {avg_prec/10:.3f}")
print(f"Avg. Recall: {avg_rec/10:.3f}")
print(f"Avg. F1Score: {avg_f1/10:.3f}")
print(f"Avg. Auc: {avg_auc/10:.3f}")
print(f"Avg. Specificity: {avg_spec/10:.3f}")
print(f"Avg. MCC: {avg_mcc/10:.3f}")
print('\n===========================================')




Linear SVM


İterasyon: 1

confusion_matrix:
 [[435  23]
 [  0  86]]

Accuracy Score: 0.958
Precision Score: 0.950
Recall Score: 1.000
F1 Score: 0.974
Auc Score: 0.894
Specificity Score: 0.789
Mcc: 0.866


İterasyon: 2

confusion_matrix:
 [[437  16]
 [  0  91]]

Accuracy Score: 0.971
Precision Score: 0.965
Recall Score: 1.000
F1 Score: 0.982
Auc Score: 0.925
Specificity Score: 0.850
Mcc: 0.906


İterasyon: 3

confusion_matrix:
 [[440  22]
 [  1  81]]

Accuracy Score: 0.958
Precision Score: 0.952
Recall Score: 0.998
F1 Score: 0.975
Auc Score: 0.892
Specificity Score: 0.786
Mcc: 0.859


İterasyon: 4

confusion_matrix:
 [[445  15]
 [  0  84]]

Accuracy Score: 0.972
Precision Score: 0.967
Recall Score: 1.000
F1 Score: 0.983
Auc Score: 0.924
Specificity Score: 0.848
Mcc: 0.906


İterasyon: 5

confusion_matrix:
 [[446  18]
 [  1  78]]

Accuracy Score: 0.965
Precision Score: 0.961
Recall Score: 0.998
F1 Score: 0.979
Auc Score: 0.905
Specificity Score: 0.812
Mcc: 0.877


İterasyon: 6

confusi